# Excercise 1: Basic Python API #

This exercise has been created by [M. Siggel](https://github.com/rainman110/tigl-workshop/commits?author=rainman110) and is part of an exercise collection on [GitHub](https://github.com/rainman110/tigl-workshop).

The goal of this excercise is to get familiar with the TiGL API and its concepts.
 - You should traverse trough the CPACS tree, get the wing shape. 
 - Create a box with OpenCASCADE
 - Export both with the TiGL IGES exporter with custom layers
 
## Loading the modules ##
As usual, load the required TiGL and TiXI modules

In [1]:
from tixi3 import tixi3wrapper
from tigl3 import tigl3wrapper
import tigl3.configuration

Now open a the CPACS file simpletest.xml using the "old" high-level python API.

In [2]:
tixi_h = tixi3wrapper.Tixi3()
tigl_h = tigl3wrapper.Tigl3()

tixi_h.open("../simpletest.cpacs.xml")
tigl_h.open(tixi_h, "")

## CPACS Traversal ##
First, get the configuration manager from tigl3.configuration and retrieve the aircraft configuration.

__Hint: CCPACSConfigurationManager_get_instance() gets the global configuration manager object__

In [3]:
# get the configuration manager
mgr = tigl3.configuration.CCPACSConfigurationManager_get_instance()

Get the aircraft config. Now, both "old" and "new" API come together, when we have to pass the TiGL handle to the configuration manager.

In [4]:
aircraft_config = mgr.get_configuration(tigl_h._handle.value)

Get the wing from the aircraft confiuration and print its wing uid. Check, what method the wing object offers.

In [5]:
wing = aircraft_config.get_wing(1)

In [6]:
print(wing.get_uid())

Wing


In [7]:
help(wing)

Help on CCPACSWing in module tigl3.configuration object:

class CCPACSWing(CPACSWing, CTiglRelativelyPositionedComponent)
 |  CCPACSWing(*args)
 |  
 |  C++ includes: CCPACSWing.h
 |  
 |  Method resolution order:
 |      CCPACSWing
 |      CPACSWing
 |      tigl3.core.CTiglReqUIDObject
 |      tigl3.core.CTiglUIDObject
 |      tigl3.core.ITiglUIDRefObject
 |      CTiglRelativelyPositionedComponent
 |      tigl3.geometry.CTiglAbstractGeometricComponent
 |      tigl3.geometry.ITiglGeometricComponent
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args)
 |      __init__(CCPACSWing self, CCPACSWings parent, CTiglUIDManager uidMgr) -> CCPACSWing
 |      
 |      Parameters
 |      ----------
 |      parent: tigl::CCPACSWings *
 |      uidMgr: tigl::CTiglUIDManager *
 |      
 |      __init__(CCPACSWing self, tigl::CCPACSRotorBlades * parent, CTiglUIDManager uidMgr) -> CCPACSWing
 |      
 |      Parameters
 |      ----------
 |      parent: tigl::CCPACSRotor

## Wing trailing edge renaming ##

Now next tasks are:
 - Get the wing shape
 - Print all face names of the wing
 - Rename the trailing edge faces

In [8]:
wing_shape = wing.get_loft()

In [9]:
for iface in range(0, wing_shape.get_face_count()):
    if wing_shape.get_face_traits(iface).name() == "TrailingEdge":
        wing_shape.get_face_traits(iface).set_name("EdgeTrail")
    print(wing_shape.get_face_traits(iface).name())

Bottom
Top
EdgeTrail
Bottom
Top
EdgeTrail
Inside
Outside


## OCC plotting ##

We use openCascade to plot the wing shape.

In [10]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [13]:
renderer = JupyterRenderer(size=(700,700))
renderer.DisplayShape(wing_shape.shape(), transparency=True)
renderer.Display()

## Create named shape with a box ##

- Create a box with OpenCASCADE. You can use the BRepPrimAPI_MakeBox class from the OCC.BRepPrimAPI package.
- Then, create a named shape from the TopoDS_Shape.

In [15]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
box = BRepPrimAPI_MakeBox(0.3, 0.8, 0.3).Solid()

In [16]:
from tigl3.geometry import CNamedShape
box_ns = CNamedShape(box, "MyBox")

## Export of the shapes ##
Finally, you have to export both shapes into the IGES format. Both shapes should have a different IGES layer!

You can inspect the result in TiGL Viewer by drag-and-dropping the igs file into the viewer.

In [17]:
from tigl3.exports import create_exporter, IgesShapeOptions
iges_exporter = create_exporter("igs")
iges_exporter.add_shape(wing_shape, IgesShapeOptions(1111))
iges_exporter.add_shape(box_ns, IgesShapeOptions(2222))
iges_exporter.write("ex1.igs")

True